In [1]:
import sys
import os
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from collections import Counter

In [2]:
sys.path.append(os.path.abspath('../library'))
import data as d
import preprocess as p
import utils as u
import bayes as b
import pipeline_npxl as npxl
import results as r
import figures as figs

# Demo on using the decoder for Neuropixel data

## 1. Load and preprocess data

### 1.1 Initialise NpxlData object

Here, we initialise a `NpxlData` object (from `data.py`) with the following attributes:

- `mouse_ID` is the name of the "mouse". You can also be specific and write the session and the brain area, etc.
- `tau` is the size of time bin in seconds. 0.2 = 200ms.
- `ms` is just converting `tau` back into `str` and in miliseconds to load the data later.
- `rewardzone` is self explnatory.

In [20]:
mouse_ID = 'VDCN09_20240729'     # e.g. can also be 'VDCN09_200ms_BSC'
tau = 0.2
ms = str(int(tau * 1000))
rewardzone = np.arange(46,62).tolist()
VDCN09 = d.NpxlData(mouse_ID, tau, rewardzone)

### 1.2 Loading data

- change areas as required
- change filepath as required

In [21]:
areas = ['midbrain', 'LP', 'vispm', 'hpf', 'DG', 'LGNd', 'BSC', 'lower', 'v1']
area = 'vispm'

data_fr_lgt = np.load("../datafiles/Neuropixel/20240729/20240729_az_VDCN09_imec0_725_"+ area +"_200ms_spike_rate.npz")
data_fr_drk = np.load("../datafiles/Neuropixel/20240729/20240729_az_VDCN09_imec0_1322_"+ area +"_200ms_spike_rate.npz")

data_count_lgt = np.load("../datafiles/Neuropixel/20240729/20240729_az_VDCN09_imec0_725_"+ area +"_200ms_spike_count.npz")
data_count_drk = np.load("../datafiles/Neuropixel/20240729/20240729_az_VDCN09_imec0_1322_"+ area +"_200ms_spike_count.npz")

In [ ]:
count_shuffled_lgt = np.load("../datafiles/Neuropixel/shuffled/20240812_az_VDCN09_imec0_light_all_spike_counts_shuffled_200ms_10cm.npz")
count_shuffled_drk = np.load("../datafiles/Neuropixel/shuffled/20240812_az_VDCN09_imec0_dark_all_spike_counts_shuffled_200ms_10cm.npz")

fr_shuffled_lgt = np.load("../datafiles/Neuropixel/shuffled/20240812_az_VDCN09_imec0_light_all_fr_shuffled_200ms_10cm.npz")
fr_shuffled_drk = np.load("../datafiles/Neuropixel/shuffled/20240812_az_VDCN09_imec0_dark_all_fr_shuffled_200ms_10cm.npz")

### 1.3 Defining the mouse

From here onwards, new attributes will be defined for a given `NpxlData` object.
To avoid manually changing it everytime we run a new mouse or new session, we will define
it here. Although it is said to be a `mouse`, again it can mean a specific sesion or
a specific brain area you are running, depending on how you initialise it earlier.

In [22]:
mouse = VDCN09

### 1.4 Preprocessing data

Pre-processing data steps:
- swapping axes to make it (Trial, Tbin, Neurons)
- defining position matrices
- correcting position matrix artifacts
- inspect trial start location (to see if there are further artifacts)


Pre-processing shuffled data steps:
- swapping axes to make it (Trial, Tbin, Neurons, Reps)

In [ ]:
# Swap axes
mouse.spikes_lgt = np.swapaxes(data_count_lgt['count'], 1,2)
mouse.spikes_drk = np.swapaxes(data_count_drk['count'], 1,2)

mouse.fr_lgt = np.swapaxes(data_fr_lgt['fr'], 1,2)
mouse.fr_drk = np.swapaxes(data_fr_drk['fr'], 1,2)

# Defining position matrix
mouse.pos_lgt = data_count_lgt['pos'][:,1,:]
mouse.pos_drk = data_count_drk['pos'][:,1,:]

print("Trial, Time Bins, Neurons")
print(mouse.spikes_lgt.shape)
print(mouse.fr_lgt.shape)
print(mouse.pos_lgt.shape)
print(mouse.spikes_drk.shape)
print(mouse.fr_drk.shape)
print(mouse.pos_drk.shape)
print(mouse.__dict__.keys())

# Inspect trial start location
trialstart_lgt = []
trialstart_drk = []

for trial in range(mouse.pos_lgt.shape[0]):
    trialstart_lgt.append(VDCN09.pos_lgt[trial,0])
for trial in range(mouse.pos_drk.shape[0]):
    trialstart_drk.append(VDCN09.pos_drk[trial,0])

start_location_lgt = np.unique(trialstart_lgt)
start_location_drk = np.unique(trialstart_drk)

print('start locations lgt:', start_location_lgt)
print(trialstart_lgt)
print()
print('start locations drk:', start_location_drk)
print(trialstart_drk)

In [72]:
# Correcting position matrix artifacts

# VDCN09 20240812 200ms
# mouse.pos_lgt[96,0] = 1
# mouse.pos_lgt[112,:2] = 16
# mouse.pos_lgt[19,0] = 16     # for 250ms bins of VDCN09

In [ ]:
# VDCN09 20240729 200ms
correct_start = [1, 4, 8, 11, 15, 18, 22, 25, 29, 32, 36]
wrong_start = [5, 12, 19, 26, 33]

wrong_trials_lgt = {}
wrong_trials_drk = {}

for pos in wrong_start:
    wrong_trials_lgt[pos] = np.where(mouse.pos_lgt[:,0] == pos)[0]
    wrong_trials_drk[pos] = np.where(mouse.pos_drk[:,0] == pos)[0]


# Mapping from wrong to correct start positions
shift_map = {5: 4, 12: 11, 19: 18, 26: 25, 33: 32}

# Correct pos_lgt
for wrong_pos, trial_indices in wrong_trials_lgt.items():
    correct_pos = shift_map[wrong_pos]
    mouse.pos_lgt[trial_indices, 0] = correct_pos

# Correct pos_drk
for wrong_pos, trial_indices in wrong_trials_drk.items():
    correct_pos = shift_map[wrong_pos]
    mouse.pos_drk[trial_indices, 0] = correct_pos

# Inspect trial start location again
trialstart_lgt = []
trialstart_drk = []

for trial in range(mouse.pos_lgt.shape[0]):
    trialstart_lgt.append(VDCN09.pos_lgt[trial,0])
for trial in range(mouse.pos_drk.shape[0]):
    trialstart_drk.append(VDCN09.pos_drk[trial,0])

start_location_lgt = np.unique(trialstart_lgt)
start_location_drk = np.unique(trialstart_drk)

print('start locations lgt:', start_location_lgt)
print(trialstart_lgt)
print()
print('start locations drk:', start_location_drk)
print(trialstart_drk)


In [ ]:
# Preprocess shuffled data

mouse.spikes_shuffled_lgt = np.swapaxes(np.swapaxes(count_shuffled_lgt['count'], 1,2), 0,1)
mouse.spikes_shuffled_drk = np.swapaxes(np.swapaxes(count_shuffled_drk['count'], 1,2), 0,1)

mouse.fr_shuffled_lgt = np.swapaxes(np.swapaxes(fr_shuffled_lgt['fr'], 1,2), 0,1)
mouse.fr_shuffled_drk = np.swapaxes(np.swapaxes(fr_shuffled_drk['fr'], 1,2), 0,1)

print("Trial, Time Bins, Neurons, Reps")
print(mouse.spikes_shuffled_lgt.shape)
print(mouse.spikes_shuffled_drk.shape)
print(mouse.fr_shuffled_lgt.shape)
print(mouse.fr_shuffled_drk.shape)

## 2. Running decoder and results

### 2.1 Running decoder in chunks of trials

Essentially, the pipeline `npxl.run_decoder_chunks()` sorts and chunks the data by
trial start locations, run them separately, then concatenate back the `posterior` and
`decoded_pos` for a given `mouse`.

Outputs are `dicts` with paradigms as keys `['lgtlgt', 'drkdrk', 'lgtdrk', 'drklgt']`, as an example:
- `posterior['lgtlgt']` has shape (Trial, Time bins, Position Bins)
- `decoded_pos['lgtlgt']` has shape (Trial, Time bins)

Key functions under the hood:
- `npxl.get_tuning_curves()` which is a wrapper for masking, smoothing (if required), and binning firing rates into spatial tuning curves.
- `u.sort_and_chunk()` for sorting and chunking data, position matrices, etc.
- `b.bayesian_decoder_chunks()` which is the main decoder function.

Properties and their default values:
- `mouse`: the NpxlData object
- `x`: the number of position bins to mask in the beginning of the tunnel. Default is `5` = 50cm.
- `tunnellength`: Used for position binning spatial tuning curves. Default is `50` position bins.
- `num_pbins`: Tunnel length excluding the reward zone. Default is `46` position bins.
- `smooth`: Whether to position bin spikes to get firingrate tuning curves. Default is `False` as spikerate is often provided.
- `SDfrac`: For computing sigma, see `u.compute_sigma()`. Default is `0.2`.
- `scale`: Whether to scale firing rates, see `u.scale_firingrate()` Default is `True`.
- `uniformprior`: If `True`, prior = 1 / num_pbins for all trials. Default is `False`, where prior = 1 / triallength.
- `discrete`: Trial start have discrete location instead of continuous (in some of Alfredo's mice). Default is `True`.
- `num_chunks`: Number of chunks that have different start locations. Default is `6`.

In [ ]:
mouse.posterior, mouse.decoded_pos = npxl.run_decoder_chunks(mouse, num_chunks=11)

### 2.2 Generating decoding results from posterior and decoded_pos

Result types:
- `'confusion_mtx'`: A 46 by 46 matrix. Vertical axis is True position, Horizontal axis is Decoded position.
- `'mean_accuracy'`: An average across trials of mean accuracy per trial.
- `'mean_error'`: An average across trials of mean error per trial.
- `'median_error'`: An average across trials of median error per trial.
- `'rt_mse'`: An average across trials of root mean squared error per trial.
- `'mean_wt_error'`: An average across trials of weighted error by posterior per trial.
- `'MostFreqPred_error'`: An average across position bins of errors between true position and most frequently decoded position.

Output are nested `dict` with result type on the first level and paradigms on the second level, examples:
- `results['confusion_mtx']['lgtlgt']` has shape (46, 46)
- `results['mean_accuracy']['lgtlgt']` is a single float
- `results['mean_error']['lgtlgt']` is a single float

Properties and their default values:
- `mouse`: the NpxlData object
- `num_chunks`: Number of chunks that have different start locations. Default is `6`.
- `num_pbins`: Tunnel length excluding the reward zone. Default is `46` position bins.
- `discrete`: Trial start have discrete location instead of continuous (in some of Alfredo's mice). Default is `True`.

In [ ]:
mouse.results = npxl.run_results_chunks(mouse, num_chunks=11)

### 2.3 Generating output DataFrame/CSV and plotting Confusion Matrices

DataFrame can be output as CSV. Change file path as required.

For confusion matrices, the figure can be saved if toggle `save=True`. Default is `False`. `filepath` default is `None`. Change it as required.

In [ ]:
# Generating output DataFrame
paradigms = ['lgtlgt', 'drkdrk', 'lgtdrk', 'drklgt']
data = {key: [mouse.results[key][p] for p in paradigms] for key in mouse.results if key != 'confusion_mtx'}
df = pd.DataFrame(data, index=paradigms)
df.index.name = 'paradigm'

# Export DataFrame to CSV
df.to_csv("../results/csv/"+ mouse.mouse_ID +"_"+ ms +"ms_"+ area +"_results.csv")
display(df)

# Plot confusion matrix
for paradigm in paradigms:
    filepath = '../results/figures/'+ mouse.mouse_ID +'_'+ ms +'ms_'+ area +'_confusion_mtx_'+ paradigm +'.png'
    # Option to save the figure. If True, save to filepath.
    figs.plot_confusion_mtx(mouse, mouse.results['confusion_mtx'][paradigm], paradigm, True, filepath)

## 3. Running Chance Estimate

### 3.1 Running decoder chance pipeline


Essentially, same as `npxl.run_decoder_chunks()` but for multiple reps. Note that `npxl.run_decoder_chunks()` is a prerequisite before running the chance pipeline.

Outputs are saved with `pickle` to save time from running it again in the future.

Outputs are `list` containing `dict` for each rep with paradigms as keys `['lgtlgt', 'drkdrk', 'lgtdrk', 'drklgt']`, as an example:
- `posterior_allreps[0]['lgtlgt']` is posterior of rep 0 with shape (Trial, Time bins, Position Bins)
- `decoded_pos_allreps[0]['lgtlgt']` is decoded_pos of rep 0 with shape (Trial, Time bins)

Additional properties on top of those in `npxl.run_decoder_chunks()`:
- `num_reps`: Default is `100`.

In [ ]:
mouse.posterior_allreps, mouse.decoded_pos_allreps = npxl.run_decoder_chance(mouse, num_chunks=6, smooth=False)

with open('../variables/'+ mouse.mouse_ID +'_'+ ms +'_posterior_allreps.pkl', 'wb') as f:
    pickle.dump(mouse.posterior_allreps, f)

with open('../variables/'+ mouse.mouse_ID +'_'+ ms +'_decoded_pos_allreps.pkl', 'wb') as f:
    pickle.dump(mouse.decoded_pos_allreps, f)

### 3.2 Running chance results pipeline

Essentially, same as `npxl.run_results_chunks()` but for multiple reps. Only difference is it does not have confusion matrices in the output.

Outputs are saved with `pickle`.

Result types:
- `'mean_accuracy_allreps'`: An average across trials of mean accuracy per trial.
- `'mean_error_allreps'`: An average across trials of mean error per trial.
- `'median_error_allreps'`: An average across trials of median error per trial.
- `'rt_mse_allreps'`: An average across trials of root mean squared error per trial.
- `'mean_wt_error_allreps'`: An average across trials of weighted error by posterior per trial.
- `'MostFreqPred_error_allreps'`: An average across position bins of errors between true position and most frequently decoded position.

Within each result type is a `list` of `dict`, examples:
- `results_allreps['mean_accuracy_allreps'][0]['lgtlgt']` is a single float
- `results['mean_error_allreps'][0]['lgtlgt']` is a single float

Additional properties on top of those in `npxl.run_results_chunks()`:
- `num_reps`: Default is `100`.

In [ ]:
mouse.results_allreps = npxl.run_results_chance(mouse, num_chunks=6)

with open('../variables/'+ mouse.mouse_ID +'_'+ ms +'_results_allreps.pkl', 'wb') as f:
    pickle.dump(mouse.results_allreps, f)